### Init

In [1]:
import torch

def count_parameters(model):
    """
    計算模型的參數數量
    Returns:
      total_params: 包含所有參數
      trainable_params: 只包含 requires_grad=True 的參數
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

import io

def estimate_model_size(model):
    """
    將模型序列化到緩衝區，估算存檔大小（MB）
    """
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size_mb = buffer.getbuffer().nbytes / (1024 ** 2)
    return size_mb

def show_network_status( orig_net, prune_net ):
    print( "原始網路參數統計:\n" )
    model = orig_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

    print( "剪枝網路參數統計:\n" )
    model = prune_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

In [2]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

In [3]:
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio

logger = setup_logger("OS2D")

In [4]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
cfg.defrost()
cfg.init.model = './src/util/checkpoints-test/checkpoint_lcp_finetune_26_layer3.5.conv2.pth'
cfg.freeze()

# net = torch.load( cfg.init.model, map_location=torch.device('cuda') )
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information.csv' )

from src.lcp.lcp import LCP
lcp = LCP(net, aux_net , dataloader_train)
lcp.init_for_indices()
lcp.set_prune_db( prune_db )

from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )

2025-07-18 17:22:44,371 OS2D INFO: Building the OS2D model
2025-07-18 17:22:44,742 OS2D INFO: Creating model on one GPU
2025-07-18 17:22:44,770 OS2D INFO: Reading model file ./src/util/checkpoints-test/checkpoint_lcp_finetune_26_layer3.5.conv2.pth
2025-07-18 17:22:44,808 OS2D INFO: Loaded feature extractor from checkpoint
2025-07-18 17:22:44,837 OS2D INFO: Restoring pruned parameters to original dimensions...
Found 40 total layers in database
Filtered to 20 layers starting with 'layer'
[LCP] 層級依賴分析完成 - net_feature_maps.layer1.0.conv1
      BatchNorm: net_feature_maps.layer1.0.bn1
      下游層級: 1 個
      跳躍連接: 0 個
[LCP] 層級依賴分析完成 - net_feature_maps.layer1.0.conv2
      BatchNorm: net_feature_maps.layer1.0.bn2
      下游層級: 1 個
      跳躍連接: 0 個
[LCP] 層級依賴分析完成 - net_feature_maps.layer1.1.conv1
      BatchNorm: net_feature_maps.layer1.1.bn1
      下游層級: 1 個
      跳躍連接: 0 個
[LCP] 層級依賴分析完成 - net_feature_maps.layer1.1.conv2
      BatchNorm: net_feature_maps.layer1.1.bn2
      下游層級: 1 個
      跳躍連接: 0

In [5]:
# from src.lcp.recontruction import LCPReconstruction
# lcp_reconstruction = LCPReconstruction(
#     prune_db = prune_db,
#     pruner = pruner,
#     prune_net = lcp._prune_net
# )

In [6]:
layers = prune_db.get_all_layers()
pruned_layers = []
for layer in layers:
    if layer not in pruned_layers and layer.startswith('layer'):
        pruned_layers.append(layer)

In [7]:
print( pruned_layers)

['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.2.conv1', 'layer1.2.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.3.conv1', 'layer2.3.conv2', 'layer3.0.conv2', 'layer3.1.conv2', 'layer3.2.conv2', 'layer3.3.conv2', 'layer3.4.conv2', 'layer3.5.conv2']


In [8]:
layers = lcp.get_layers_name()

for name, ch in layers:
    if name == 'layer1.0.conv1':
        pass
    else:
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")

layer1.0.conv1: 64 channels
[LCP] 開始基於數學推導的無梯度通道重要性計算 - net_feature_maps.layer1.0.conv1
[322, 372, 298]
[LOG] 原始網路特徵提取完成
[LOG] 剪枝網路特徵提取完成
322 {'channels': [{'l1_norm': 3.1238338947296143, 'variance': 0.19176186621189117, 'mean_deviation': 2.702387809753418, 'energy': 9.95009708404541, 'sparsity': 0.0, 'importance': 1.5046344995498657}, {'l1_norm': 2.974565267562866, 'variance': 0.38810089230537415, 'mean_deviation': 2.55311918258667, 'energy': 9.236136436462402, 'sparsity': 0.0, 'importance': 1.4393484592437744}, {'l1_norm': 0.0, 'variance': 0.0, 'mean_deviation': 0.4214460849761963, 'energy': 0.0, 'sparsity': 1.0, 'importance': 0.04843650385737419}, {'l1_norm': 0.0, 'variance': 0.0, 'mean_deviation': 0.4214460849761963, 'energy': 0.0, 'sparsity': 1.0, 'importance': 0.04843650385737419}, {'l1_norm': 3.0585670471191406, 'variance': 10.141693115234375, 'mean_deviation': 1.9314370155334473, 'energy': 15.677680015563965, 'sparsity': 0.0, 'importance': 2.345611810684204}, {'l1_norm': 0.0, '

In [9]:
for layer in pruned_layers:
    lcp.prune_layer(
        layer_name   = layer,
        discard_rate = None,
    )

[PRUNE] 開始剪枝層級: layer1.0.conv1
=== net_feature_maps 詳細層級資訊 ===
[LCP] 層級依賴分析完成 - layer1.0.conv1
      BatchNorm: net_feature_maps.layer1.0.bn1
      下游層級: 1 個
      跳躍連接: 0 個
[PRUNE] 開始剪枝輸出通道: layer1.0.conv1
[PRUNE] 保留通道索引: [0, 1, 4, 7, 13, 14, 24, 29, 44, 54, 60, 62, 63]
[PRUNE] 保留通道數量: 13
[PRUNE] 原始輸出通道數: 64
[PRUNE] Conv2d 權重: torch.Size([64, 64, 1, 1]) -> torch.Size([13, 64, 1, 1])
[PRUNE] 更新 out_channels: 13
[PRUNE] 輸出通道剪枝完成: 64 -> 13
[PRUNE] 開始剪枝 BatchNorm 層: net_feature_maps.layer1.0.bn1
[PRUNE] BatchNorm 通道數變化: 64 -> 13
[PRUNE] BatchNorm weight 剪枝完成: torch.Size([13])
[PRUNE] BatchNorm bias 剪枝完成: torch.Size([13])
[PRUNE] BatchNorm running_mean 剪枝完成: torch.Size([13])
[PRUNE] BatchNorm running_var 剪枝完成: torch.Size([13])
[VALIDATE] BatchNorm 層 net_feature_maps.layer1.0.bn1 剪枝驗證通過
[PRUNE] BatchNorm 層 net_feature_maps.layer1.0.bn1 剪枝成功
[PRUNE] 修補 net_feature_maps.layer1.0.conv2 的輸入通道 (來源 layer1.0.conv1)
[PRUNE] Conv 輸入維度: torch.Size([64, 64, 3, 3]) → torch.Size([64, 13, 3, 3])
[0, 1, 4

In [10]:
show_network_status( net, lcp._prune_net )

原始網路參數統計:

總參數量: 10,169,478
可訓練參數量: 10,169,478
模型存儲大小: 39.05 MB
剪枝網路參數統計:

總參數量: 6,997,533
可訓練參數量: 6,997,533
模型存儲大小: 26.93 MB


In [11]:
# for i in range( 20 ):
#     lcp.debug_for_test_vision(
#         dataloader_train = dataloader_train,
#         img_normalization = img_normalization,
#         box_coder = box_coder,
#         cfg = cfg,
#         count = 1
#     )

In [12]:
# 基本評估
results = lcp.evaluate(dataloader_train, img_normalization, box_coder, cfg, count=50)

# # 保存結果
# lcp.save_evaluation_results(results, "pruned_model_evaluation.json")

# # 可視化單個樣本
# lcp.visualize_evaluation_sample(dataloader_train, img_normalization, box_coder, cfg)

# # 使用不同的IoU閾值進行評估
# results_strict = lcp.evaluate(dataloader_train, img_normalization, box_coder, cfg, 
#                                    count=100, iou_threshold=0.7)


開始評估 50 張圖像...
Image 56 size FeatureMapSize(w=3264, h=2448) has 17 boxes


c:\Users\marti\anaconda3\envs\ntut-project\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[((0.05505147321670215, 0.0), (0.36141354602699194, 0.43856912543903964)), ((0.6738199493901559, 0.6073022479732005), (0.9154952051457852, 1.0)), ((0.0, 0.6088683941072459), (0.19607413558431366, 0.9831397311333238)), ((0.1708903323695511, 0.617684943571377), (0.3717804307475101, 0.9854671487081409))]
converted_boxes: [((0.05505147321670215, 0.0), (0.36141354602699194, 0.43856912543903964)), ((0.6738199493901559, 0.6073022479732005), (0.9154952051457852, 1.0)), ((0.0, 0.6088683941072459), (0.19607413558431366, 0.9831397311333238)), ((0.1708903323695511, 0.617684943571377), (0.3717804307475101, 0.9854671487081409))], tpoints: [((0.02499999953251259, 0.6767000185897927), (0.18999999177222157, 0.950000039892259)), ((0.1875, 0.6800000059838388), (0.35000000747979854, 0.9600000319138072)), ((0.01999999962601007, 0.42330000135633683), (0.1875, 0.689999998005387)), ((0.1875, 0.43000000598383886), (0.3449999865363626, 0.6932999853994332))]
Image 421 size FeatureMapSize(w=2448, h=3264) has 9 bo

In [ ]:
import os
import csv
import json
from datetime import datetime

def save_evaluation_to_csv(results, filename='evaluation_results.csv'):
    """
    將評估結果保存到 CSV 文件
    
    Args:
        results: 評估結果字典
        filename: CSV 文件名
    """
    
    # 確保目錄存在
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    
    # 提取主要指標
    overall_metrics = results['overall_metrics']
    bijection_stats = results['bijection_stats']
    eval_params = results['evaluation_params']
    
    # 計算額外統計
    class_metrics = results['class_metrics']
    num_classes = len(class_metrics)
    classes_with_perfect_score = sum(1 for metrics in class_metrics.values() if metrics['f1_score'] == 1.0)
    classes_with_zero_score = sum(1 for metrics in class_metrics.values() if metrics['f1_score'] == 0.0)
    
    # 準備 CSV 數據
    csv_data = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_type': eval_params['model_type'],
        'evaluated_images': eval_params['evaluated_images'],
        'iou_threshold': eval_params['iou_threshold'],
        'precision': overall_metrics['precision'],
        'recall': overall_metrics['recall'],
        'f1_score': overall_metrics['f1_score'],
        'mean_iou': overall_metrics['mean_iou'],
        'map': overall_metrics['map'],
        'matching_rate': overall_metrics['matching_rate'],
        'total_detections': bijection_stats['total_detections'],
        'total_ground_truths': bijection_stats['total_ground_truths'],
        'total_matched_pairs': bijection_stats['total_matched_pairs'],
        'unmatched_detections': bijection_stats['unmatched_detections'],
        'unmatched_gts': bijection_stats['unmatched_gts'],
        'num_classes': num_classes,
        'classes_perfect_score': classes_with_perfect_score,
        'classes_zero_score': classes_with_zero_score
    }
    
    # 檢查文件是否存在以決定是否寫入標題
    file_exists = os.path.exists(filename)
    
    # 寫入 CSV
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_data.keys())
        
        # 如果文件不存在，寫入標題
        if not file_exists:
            writer.writeheader()
        
        # 寫入數據
        writer.writerow(csv_data)
    
    print(f"評估結果已保存到: {filename}")
    return filename

# 使用範例
filename = save_evaluation_to_csv(results)


評估結果已保存到: evaluation_results.csv
